In [6]:
!pip install music21

In [7]:
from PIL import Image
import numpy as np
from music21 import instrument, note, chord, stream

lowerBoundNote = 21
def column2notes(column):
    notes = []
    for i in range(len(column)):
        if column[i] > 255/2:
            notes.append(i+lowerBoundNote)
    return notes

resolution = 0.25
def updateNotes(newNotes,prevNotes): 
    res = {} 
    for note in newNotes:
        if note in prevNotes:
            res[note] = prevNotes[note] + resolution
        else:
            res[note] = resolution
    return res

def image2midi(image_path):
    with Image.open(image_path) as image:
        im_arr = np.frombuffer(image.tobytes(), dtype=np.uint8)
        try:
            im_arr = im_arr.reshape((image.size[1], image.size[0]))
        except:
            im_arr = im_arr.reshape((image.size[1], image.size[0],3))
            im_arr = np.dot(im_arr, [0.33, 0.33, 0.33])

    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model

    prev_notes = updateNotes(im_arr.T[0,:],{})
    for column in im_arr.T[1:,:]:
        notes = column2notes(column)
        # pattern is a chord
        notes_in_chord = notes
        old_notes = prev_notes.keys()
        for old_note in old_notes:
            if not old_note in notes_in_chord:
                new_note = note.Note(old_note,quarterLength=prev_notes[old_note])
                new_note.storedInstrument = instrument.Piano()
                if offset - prev_notes[old_note] >= 0:
                    new_note.offset = offset - prev_notes[old_note]
                    output_notes.append(new_note)
                elif offset == 0:
                    new_note.offset = offset
                    output_notes.append(new_note)                    
                else:
                    print(offset,prev_notes[old_note],old_note)

        prev_notes = updateNotes(notes_in_chord,prev_notes)

        # increase offset each iteration so that notes do not stack
        offset += resolution

    for old_note in prev_notes.keys():
        new_note = note.Note(old_note,quarterLength=prev_notes[old_note])
        new_note.storedInstrument = instrument.Piano()
        new_note.offset = offset - prev_notes[old_note]

        output_notes.append(new_note)

    prev_notes = updateNotes(notes_in_chord,prev_notes)

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=image_path.split("/")[-1].replace(".png",".mid"))

In [8]:
import os

In [9]:
os.getcwd()

'/home/ec2-user/SageMaker'

In [11]:
# image_path = sys.argv[1]
# for i in range(10):
#     image2midi(os.getcwd() + '/04_16_attempt_1_classical_music_10_epochs_OutputNumber_'+str(i) + '.png')

In [6]:
# 04_18_attempt_2_classical_music_input_are_transformed_songs_10_epochs_OutputNumber_0.mid
# for i in range(10):
#     image2midi(os.getcwd() + '/04_18_attempt_2_classical_music_input_are_transformed_songs_10_epochs_OutputNumber_'+ str(i) + '.png')

In [7]:
# # 04_18_attempt_2_rock_music_input_are_transformed_songs_10_epochs_OutputNumber_0.png
# for i in range(10):
#     image2midi(os.getcwd() + '/04_18_attempt_2_rock_music_input_are_transformed_songs_10_epochs_OutputNumber_'+ str(i) + '.png')

In [5]:
# 04_18_attempt_2_jazz_music_input_are_transformed_songs_10_epochs_OutputNumber_0.png

# for i in range(10):
#     image2midi(os.getcwd() + '/04_18_attempt_2_jazz_music_input_are_transformed_songs_10_epochs_OutputNumber_'+ str(i) + '.png')

In [8]:
# 04_18_attempt_2_classical_music_input_are_transformed_songs_10_epochs_OutputNumber_0.png

# for i in range(10):
#     image2midi(os.getcwd() + '/04_18_attempt_2_classical_music_input_are_transformed_songs_10_epochs_OutputNumber_'+ str(i) + '.png')
    

    
# for i in range(10):
#     os.rename(os.getcwd() + '/04_18_attempt_2_classical_music_input_are_transformed_songs_10_epochs_OutputNumber_'+ str(i) + '.png', \
#              os.getcwd() + '/04_21_attempt_3_classical_music_overfit_OutputNumber_'+ str(i) + '.png')
#     os.rename(os.getcwd() + '/04_18_attempt_2_classical_music_input_are_transformed_songs_10_epochs_OutputNumber_'+ str(i) + '.mid', \
#          os.getcwd() + '/04_21_attempt_3_classical_music_overfit_OutputNumber_'+ str(i) + '.mid')

In [5]:
# 04_21_attempt_3_rock_overfit_OutputNumber_0.png

# for i in range(10): 
#     image2midi(os.getcwd() + '/04_21_attempt_3_rock_overfit_OutputNumber_' + str(i) + '.png')

In [10]:
# 04_21_attempt_3_jazz_overfit_OutputNumber_0.png

for i in range(10): 
    image2midi(os.getcwd() + '/04_21_attempt_3_jazz_overfit_OutputNumber_' + str(i) + '.png')